In [4]:
import datetime as dt
from db import connection as _conn
import requests 
import pandas as pd
import io
from math import pi
import pandas as pd
from bokeh.plotting import figure, show, output_file
import matplotlib.pyplot as plt

In [5]:

def send_request_api(url):

    headers = {'content-type': 'application/json; charset=utf-8'}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise f'Falha na chamada da API. Erro {response.status_code}'


def get_df(url):


    dataJson = send_request_api(url)
    engine = _conn.conn_engine()
    listPeriodStart = []
    listPeriodEnd = []
    listPriceLow = []
    listPriceHigh = []
    listPriceOpen = []
    listPriceClose = []
    dataDict= {}
    dfBtc = None

    for i in range(len(dataJson)) :
        
        periodStartApi = dt.datetime.strptime(dataJson[i]['time_period_start'][:19], '%Y-%m-%dT%H:%M:%S')
        periodEndApi = dt.datetime.strptime(dataJson[i]['time_period_end'][:19], '%Y-%m-%dT%H:%M:%S')
        listPeriodStart.append(str(dt.datetime.strftime(periodStartApi, '%Y-%m-%d %H:%M:%S')))
        listPeriodEnd.append(str(dt.datetime.strftime(periodEndApi, '%Y-%m-%d %H:%M:%S')))
        listPriceLow.append(float(dataJson[i]['price_low']))
        listPriceHigh.append(float(dataJson[i]['price_high']))
        listPriceOpen.append(float(dataJson[i]['price_open']))
        listPriceClose.append(float(dataJson[i]['price_close']))

    dfBtc = pd.DataFrame({'PriceOpen':  listPriceOpen,
     'PriceHigh': listPriceHigh,
     'PriceLow': listPriceLow,
     'PriceClose': listPriceClose,
     'PeriodStart': listPeriodStart,
     'PeriodEnd': listPeriodEnd})

    dfBtc.to_sql('BitCoin', con=engine, if_exists='replace', index=False)
    print("Inserção no banco de dados concluída !")
    
    return dfBtc

def graphic_thirty_days(url):
    dfBtc = get_df(url)
    print(dfBtc)
    dfBtc["PeriodEnd"] = pd.to_datetime(dfBtc["PeriodEnd"])
    dfBtc["PeriodStart"] = pd.to_datetime(dfBtc["PeriodStart"])

    inc = dfBtc.PriceClose > dfBtc.PriceOpen
    dec = dfBtc.PriceOpen > dfBtc.PriceClose
    w = 12*60*60*1000

    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

    p = figure(
        x_axis_type="datetime", 
        tools=TOOLS, 
        plot_width=1000,
        sizing_mode="stretch_width",
        height=250,

        title = "Visão Bitcoin 30 dias")
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.5

    p.segment(dfBtc.PeriodStart, dfBtc.PriceHigh, dfBtc.PeriodStart, dfBtc.PriceLow,  color="black")

    p.vbar(dfBtc.PeriodStart[inc], w, dfBtc.PriceOpen[inc], dfBtc.PriceClose[inc], fill_color="#D5E1DD", line_color="black")

    p.vbar(dfBtc.PeriodStart[dec], w, dfBtc.PriceOpen[dec], dfBtc.PriceClose[dec], fill_color="#F2583E", line_color="black")


    output_file("btcCandlestick.html", title="Visão Bitcoin 30 dias")

    show(p)


def graphic_five_years(url):
    dfBtc = get_df(url)
    print(dfBtc)
    plt.title('Bitcoin em 5 anos')
    plt.plot(pd.to_datetime(dfBtc['PeriodEnd']).dt.strftime('%Y'), dfBtc['PriceClose'])
    plt.xticks(rotation=30, ha='right')
    
    # Giving x and y label to the graph
    plt.xlabel('Anos')
    plt.ylabel('Preço fechado')
    plt.show()




In [6]:
print('Escolha o processamento:\n1 - Visão Bitcoin 5 anos \n2 - Visão Bitcoin 30 dias')
tipoProcessamento = int(input("Qual o processamento ?"))


Escolha o processamento:
1 - Visão Bitcoin 5 anos 
2 - Visão Bitcoin 30 dias


In [7]:
if (tipoProcessamento == 1):
    url = 'https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BTC_USD/history?'\
            'period_id=1MTH&time_start=2018-01-01T00:00:00&apikey=B6646174-0B59-4C3F-BBF4-138AAC83EEF2'
    graphic_five_years(url)
elif(tipoProcessamento == 2):
    url = 'https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BTC_USD/history?'\
            'period_id=1DAY&time_start=2023-08-01T00:00:00&apikey=B6646174-0B59-4C3F-BBF4-138AAC83EEF2'
    graphic_thirty_days(url)

Inserção no banco de dados concluída !
    PriceOpen  PriceHigh  PriceLow  PriceClose          PeriodStart  \
0     29221.0    29715.0   28574.0     29696.0  2023-08-01 00:00:00   
1     29697.0    30025.0   28929.0     29164.0  2023-08-02 00:00:00   
2     29157.0    29395.0   28925.0     29176.0  2023-08-03 00:00:00   
3     29175.0    29306.0   28784.0     29077.0  2023-08-04 00:00:00   
4     29075.0    29107.0   28950.0     29049.0  2023-08-05 00:00:00   
5     29049.0    29159.0   28962.0     29042.0  2023-08-06 00:00:00   
6     29041.0    29244.0   28671.0     29181.0  2023-08-07 00:00:00   
7     29186.0    30210.0   29107.0     29764.0  2023-08-08 00:00:00   
8     29776.0    30119.0   29352.0     29565.0  2023-08-09 00:00:00   
9     29565.0    29703.0   29294.0     29423.0  2023-08-10 00:00:00   
10    29424.0    29525.0   29222.0     29398.0  2023-08-11 00:00:00   
11    29398.0    29465.0   29350.0     29415.0  2023-08-12 00:00:00   
12    29415.0    29443.0   29254.0    